# Background

Here we want to sample from $X \sim (N(1, I_3) | \ell(X) = d, X \geq 0)$ where
$$\ell(x) = \sum_{i=1}^4 u_i (r - \sum_{j=1}^{i-1} x_i)^+$$ 
with parameters $r > 0$ and $u_i, i=1, \ldots 4$ and a backwards summation assumed to be zero.  This function arises when modeling root growth in a piecewise linear fashion.

# Import

In [ ]:
import os
import numpy as np
import importlib as il

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Make sure this is set to the repository base directory
os.getenv("PYTHONPATH", "")

In [ ]:
il.import_module("ctgauss")

In [ ]:
from ctgauss import IsotropicCTGauss, AnisotropicCTGauss

# Define geometry of subspace

In [ ]:
def delta_x_regions(K, r):
    # K is number of kinks
    # r is radius
    # x_i = c_i' delta_x, i=0...,K+1, c_i = first i coords 1, else 0                                                                                                                                                                  
    # regions                                                                                                                                                                                                                         
    E = np.eye(K)
    Cp = np.tril(np.ones((K+1, K+1)), -1)[:,0:-1] # K+1 x K: C[0,:] is unnecessary, but makes indexing easier                                                                                                                         
    Cn = Cp
    gE = np.full((K,), fill_value=0.)
    gCp = np.full((K+1,), fill_value=-r)
    gCn = np.full((K+1,), fill_value=-r)
    sE = np.zeros((K+1, K))
    sCp = np.zeros((K+1, K+1))
    sCn = np.zeros((K+1, K+1))
    # This is if we allow x_1 > r (fortran indexing)                                                                                                                                                                                  
    # Region 1                                                                                                                                                                                                                        
    k_f, k_c = 1, 0
    sCp[k_c, k_f] = k_f + 1
    sE[k_c,:] = k_f
    # Region 2..K                                                                                                                                                                                                                     
    for k_f in range(2, K+1):
        k_c = k_f - 1
        sCp[k_c, k_f] = k_f + 1 # move up                                                                                                                                                                                             
        sCn[k_c, k_f - 1] = -(k_f - 1) # move down                                                                                                                                                                                    
        sE[k_c,:] = k_f
    # Region K+1                                                                                                                                                                                                                      
    k_f, k_c = K+1, K
    sCn[k_c, k_f - 1] = -(k_f - 1) # move down                                                                                                                                                                                        
    sE[k_c,:] = k_f
    # If we do not allow x_1 > r, then we must set                                                                                                                                                                                    
    # k_f, k_c = 2, 1                                                                                                                                                                                                                 
    sCn[1, 1] = -2 # reflect back into region 2                                                                                                                                                                                       
    sCp[0, 1] = 1 # reflect back into region 1, but should never be in region 1                                                                                                                                                       
    F = np.vstack([Cp, Cn, E])
    g = np.concatenate([gCp, gCn, gE], axis=None)
    L = np.hstack([sCp, sCn, sE])
    return (F, g, L)

In [ ]:
def delta_x_constraints(dv, r, h):
    # v_i = sum(dv[0:i]), so v_0 = 0
    K = dv.shape[0] - 1
    C = np.tril(np.ones((K+1, K+1)), 0)
    # m = C delta_m                                                                                                                                                                                                                   
    v = np.matmul(C, dv)
    A = np.zeros((K+1, K, 1))
    b = np.zeros((K+1,))
    # We should not have to use region 1 (0 in c-indexing)                                                                                                                                                                                              
    for k_c in range(1, K+1):
        # k_f = k_c + 1
        A[k_c,0:k_c,0] = v[0:k_c] - v[k_c]                                                                                                                                                                                         
    b = - (h - (v * r).reshape(K+1, 1)) # y from A'x + y
    return (A, b)

In [ ]:
K = 3

In [ ]:
r = 4

In [ ]:
d = 1.0

In [ ]:
dv = np.array([0.15, 0.20, 0.25, 0.30])

In [ ]:
F, g, L = delta_x_regions(K, r)

In [ ]:
A, y = delta_x_constraints(dv, r, d)

# Plot

In [ ]:
def make_plane_mesh(a, b, independent_axes, dependent_axis, bounds1, bounds2, ngrid):
    i1, i2 = independent_axes
    i3 = dependent_axis
    grid1 = np.linspace(bounds1[0], bounds1[1], ngrid)
    grid2 = np.linspace(bounds2[0], bounds2[1], ngrid)
    M1, M2 = np.meshgrid(grid1, grid2)
    mesh_shape = M1.shape
    M3 = - ((a[i1] * M1 + a[i2] * M2) + b) / a[i3]
    cube_grid = np.zeros((3, mesh_shape[0], mesh_shape[1]))
    cube_grid[i1] = M1
    cube_grid[i2] = M2
    cube_grid[i3] = M3
    return (cube_grid[0], cube_grid[1], cube_grid[2])

In [ ]:
Ftil = np.vstack((F[5:11,:], A.squeeze()[1:,:]))

In [ ]:
gtil = np.hstack((g[5:11], y.squeeze()[1:]))

In [ ]:
# Concatenation of F and A and g and y give us the information we need to make the planes
Ftil, g

In [ ]:
xp = np.zeros(Ftil.shape)

In [ ]:
# Make normal vectors to planes
ntimes = 3
times = np.linspace(0, 1.1, ntimes)
XYZ = np.zeros((Ftil.shape[0], Ftil.shape[1], ntimes))
for i in range(Ftil.shape[0]):
    f = Ftil[i,:]
    xp_i = -gtil[i] * f / np.dot(f, f)
    xp[i,:] = xp_i
    XYZ[i,:,:] = np.outer(f, times) + xp_i.reshape(3,1)

In [ ]:
# Make planes

In [ ]:
bounds = (0, 4)
ngrid = 10

In [ ]:
# Boundaries for reflecting - index corresponds to fortran index of row in Ftil

In [ ]:
xx4, yy4, zz4 = make_plane_mesh(np.array([1, 0, 0]), 0, (1, 2), 0, bounds, bounds, ngrid)

In [ ]:
xx5, yy5, zz5 = make_plane_mesh(np.array([0, 1, 0]), 0, (0, 2), 1, bounds, bounds, ngrid)

In [ ]:
xx6, yy6, zz6 = make_plane_mesh(np.array([0, 0, 1]), 0, (0, 1), 2, bounds, bounds, ngrid)

In [ ]:
# Interior boundaries dividing region

In [ ]:
xx1, yy1, zz1 = make_plane_mesh(Ftil[0], gtil[0], (1, 2), 0, bounds, bounds, ngrid)

In [ ]:
xx2, yy2, zz2 = make_plane_mesh(Ftil[1], gtil[1], (1, 2), 0, bounds, bounds, ngrid)

In [ ]:
xx3, yy3, zz3 = make_plane_mesh(Ftil[2], gtil[2], (0, 1), 2, bounds, bounds, ngrid)

In [ ]:
# Subspace within each region

In [ ]:
xx7, yy7, zz7 = make_plane_mesh(Ftil[6], gtil[6], (1, 2), 0, bounds, bounds, ngrid)

In [ ]:
xx8, yy8, zz8 = make_plane_mesh(Ftil[7], gtil[7], (1, 2), 0, bounds, bounds, ngrid)

In [ ]:
xx9, yy9, zz9 = make_plane_mesh(Ftil[8], gtil[8], (1, 2), 0, bounds, bounds, ngrid)

In [ ]:
# xlim = np.min(XYZ[:,0,:]), np.max(XYZ[:,0,:])
# ylim = np.min(XYZ[:,1,:]), np.max(XYZ[:,1,:])
# zlim = np.min(XYZ[:,2,:]), np.max(XYZ[:,2,:])

In [ ]:
fig = plt.figure(figsize=(12, 5))
plt.suptitle("Bounding and subspace planes")
plt.title("Grey (hard boundary), Orange (boundary between regions), Blue (subspace)")
plt.box(on=None)
plt.axis('off')

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
ax1.view_init(elev=30, azim=50)
ax1.plot_surface(xx4, yy4, zz4, color="grey", alpha=0.2)
ax1.plot_surface(xx5, yy5, zz5, color="grey", alpha=0.2)
ax1.plot_surface(xx6, yy6, zz6, color="grey", alpha=0.2)
ax1.plot_surface(xx1, yy1, zz1, color="orange", alpha=0.2)
ax1.plot_surface(xx2, yy2, zz2, color="orange", alpha=0.2)
ax1.plot_surface(xx7, yy7, zz7, color="blue", alpha=0.2)
ax1.set_xlim((0,4))
ax1.set_ylim((0,4)) 
ax1.set_zlim((0,4)) 
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")
ax1.set_title("Region 1")

ax2 = fig.add_subplot(1, 3, 2, projection='3d')
ax2.view_init(elev=30, azim=50)
ax2.plot_surface(xx4, yy4, zz4, color="grey", alpha=0.2)
ax2.plot_surface(xx5, yy5, zz5, color="grey", alpha=0.2)
ax2.plot_surface(xx6, yy6, zz6, color="grey", alpha=0.2)
ax2.plot_surface(xx2, yy2, zz2, color="orange", alpha=0.2)
ax2.plot_surface(xx3, yy3, zz3, color="orange", alpha=0.2)
ax2.plot_surface(xx8, yy8, zz8, color="blue", alpha=0.2)
ax2.set_xlim((0,4))
ax2.set_ylim((0,4)) 
ax2.set_zlim((0,4)) 
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("z")
ax2.set_title("Region 2")


ax3 = fig.add_subplot(1, 3, 3, projection='3d')
ax3.view_init(elev=30, azim=50)
ax3.plot_surface(xx4, yy4, zz4, color="grey", alpha=0.2)
ax3.plot_surface(xx5, yy5, zz5, color="grey", alpha=0.2)
ax3.plot_surface(xx6, yy6, zz6, color="grey", alpha=0.2)
ax3.plot_surface(xx3, yy3, zz3, color="orange", alpha=0.2)
ax3.plot_surface(xx9, yy9, zz9, color="blue", alpha=0.2)
ax3.set_xlim((0,4))
ax3.set_ylim((0,4)) 
ax3.set_zlim((0,4)) 
ax3.set_xlabel("x")
ax3.set_ylabel("y")
ax3.set_zlabel("z")
ax3.set_title("Region 3");

# fig.savefig("positive-part-regions.png", dpi=300, pad_inches=0)

In [ ]:
fig = plt.figure()
plt.suptitle("Bounding and subspace planes (and normal vectors) for all regions")
plt.title("Grey (hard boundary), Orange (boundary between regions), Blue (subspace)")
plt.box(on=None)
plt.axis('off')

ax = fig.add_subplot(projection='3d')
ax.view_init(elev=30, azim=50)
for i in range(XYZ.shape[0]):
    ax.plot3D(XYZ[i,0,:], XYZ[i,1,:], XYZ[i,2,:], color="black", alpha=0.5)
ax.scatter(XYZ[:,0,0], XYZ[:,1,0], XYZ[:,2,0], color="green", alpha=0.5, s=100)
ax.scatter(XYZ[:,0,-1], XYZ[:,1,-1], XYZ[:,2,-1], color="red", alpha=0.5, s=100)
ax.plot_surface(xx4, yy4, zz4, color="grey", alpha=0.2)
ax.plot_surface(xx5, yy5, zz5, color="grey", alpha=0.2)
ax.plot_surface(xx6, yy6, zz6, color="grey", alpha=0.2)
ax.plot_surface(xx1, yy1, zz1, color="orange", alpha=0.2)
ax.plot_surface(xx2, yy2, zz2, color="orange", alpha=0.2)
ax.plot_surface(xx3, yy3, zz3, color="orange", alpha=0.2)
ax.plot_surface(xx7, yy7, zz7, color="blue", alpha=0.2)
ax.plot_surface(xx8, yy8, zz8, color="blue", alpha=0.2)
ax.plot_surface(xx9, yy9, zz9, color="blue", alpha=0.2)
ax.set_xlim((0,4))
ax.set_ylim((0,4)) 
ax.set_zlim((0,4)) 
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z");

# Example - Isotropic Gaussian

In [ ]:
mu = np.full((3,), fill_value=1.0)
phi = 1.0

N = 1000
t_max = 0.5*np.pi
x0 = np.array([2., 3., 2.])
x0dot = np.array([0., -1., 0.]) / 2
reg, j = 2, 1 # f vs. c indexing  

In [ ]:
rng = np.random.default_rng()

In [ ]:
ictg = IsotropicCTGauss(phi, mu, A, y, F, g, L)

In [ ]:
# DO WE PASS COTINUITY ERROR
ce = ictg.continuity_error()
np.all(ce < 1e15)

In [ ]:
(X, Xdot, R) = ictg.sample(rng, N, t_max, reg, x0, x0dot)
# (X, Xdot, R, I) = ictg.sample_with_boundaries(rng, N, t_max, reg, x0, x0dot)

In [ ]:
fig = plt.figure(figsize=(12, 5))
plt.suptitle(r"Sample of $N(1, I_3)$ constrained to subspace")
plt.title("Region 1 (Purple), Region 2 (Teal), Region 3 (Yellow)")
plt.box(on=None)
plt.axis('off')

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax1.view_init(elev=30, azim=50)
ax1.scatter3D(X[:,0], X[:,1], X[:,2], c=R, alpha=0.4, s=2)
ax1.set_xlim((0,4))
ax1.set_ylim((0,4)) 
ax1.set_zlim((0,4))
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("z")

ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.view_init(elev=20, azim=-60)
ax2.scatter3D(X[:,0], X[:,1], X[:,2], c=R, alpha=0.4, s=2)
ax2.set_xlim((0,4))
ax2.set_ylim((0,4)) 
ax2.set_zlim((0,4))
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("z");

# fig.savefig("positive-part-samples.png", dpi=300, pad_inches=0)